## Module 1 - Getting Started

This is step-by-step follow along based on Google's Tensorflow Tutorial.

[Getting Started With TensorFlow](https://www.tensorflow.org/get_started/get_started)

In [8]:
import tensorflow as tf
import platform

print("TensorFlow version: ", tf.__version__)
print("Pyhton version: ", platform.python_version())



TensorFlow version:  1.1.0
Pyhton version:  3.5.2


## Symbolic Separation - Graph vs Execution
Notice that printing the nodes does not output the values 3.0 and 4. Instead, they are nodes that, when evaluated, would produce 3.0 and 4.0, respectively. To actually evaluate the nodes, we must run the computational graph within a session. 

In [10]:
import tensorflow as tf

node1 = tf.constant(3.0, tf.float32)
node2 = tf.constant(4.0) # also tf.float32 implicitly

print(node1, node2)

sess = tf.Session()
print(sess.run([node1, node2]))

Tensor("Const_12:0", shape=(), dtype=float32) Tensor("Const_13:0", shape=(), dtype=float32)
[3.0, 4.0]


## Placeholder and Feeder
A graph can be parameterized to accept external inputs, known as placeholders. A placeholder is a promise to provide a value later.

We can evaluate this graph with multiple inputs by using the feed_dict parameter to specify Tensors that provide concrete values to these placeholders:

In [12]:
import tensorflow as tf

a = tf.placeholder(tf.float32)
b = tf.placeholder(tf.float32)
adder_node = a + b  # + provides a shortcut for tf.add(a, b)

# Placeholders allow multiple inputs to be feed into the same graph
print(sess.run(adder_node, {a: 3, b:4.5}))
print(sess.run(adder_node, {a: [1,3], b: [2, 4]}))
print(sess.run(adder_node, {a: [[1,3],[5,6]], b: [[2, 4],[8,6]]}))


7.5
[ 3.  7.]
[[  3.   7.]
 [ 13.  12.]]


In [14]:
import tensorflow as tf

a = tf.placeholder(tf.float32)
b = tf.placeholder(tf.float32)
adder_node = a + b  # + provides a shortcut for tf.add(a, b)
add_and_triple = adder_node * 3.

print(sess.run(add_and_triple, {a: 3, b:4.5}))
print(sess.run(add_and_triple, {a: [1,3], b: [2, 4]}))
print(sess.run(add_and_triple, {a: [[1,3],[5,6]], b: [[2, 4],[8,6]]}))


22.5
[  9.  21.]
[[  9.  21.]
 [ 39.  36.]]


## Variables

In machine learning we will typically want a model that can take arbitrary inputs, such as x. And to make the model trainable, we need to be able to modify the graph to get new outputs with the same input. Variables allow us to add trainable parameters (W and b) to a graph and they are constructed with a type and initial value.

Constants are initialized when you call tf.constant, and their value can never change. Variables are not initialized when you call tf.Variable. To initialize all the variables in a TensorFlow program, you must explicitly call the operation  tf.global_variables_initializer().

In [15]:
import tensorflow as tf

W = tf.Variable([.3], tf.float32)
b = tf.Variable([-.3], tf.float32)
x = tf.placeholder(tf.float32)
linear_model = W * x + b

init = tf.global_variables_initializer()

sess.run(init)
print(sess.run(linear_model, {x:[1,2,3,4]}))


[ 0.          0.30000001  0.60000002  0.90000004]


## Loss Function

To evaluate the model on training data, we need a y placeholder to provide the truth labels, and we need to write a loss function.

A loss function measures how far apart the current model is from the truth labels. 

In [16]:
import tensorflow as tf

y = tf.placeholder(tf.float32)  # Placeholder for truth labels

squared_deltas = tf.square(linear_model - y)   # loss function
loss = tf.reduce_sum(squared_deltas)

print(sess.run(loss, {x:[1,2,3,4], y:[0,-1,-2,-3]}))


23.66


## Optimizer

The whole point of machine learning is to find the correct model parameters automatically. 

TensorFlow provides optimizers that slowly change each variable in order to minimize the loss function. The simplest optimizer is gradient descent. 


In [17]:
import tensorflow as tf

optimizer = tf.train.GradientDescentOptimizer(0.01)  # The good ol' SGD
train = optimizer.minimize(loss)

sess.run(init) # reset values to incorrect defaults.

# Run SGD for 1000 steps
for i in range(1000):
  sess.run(train, {x:[1,2,3,4], y:[0,-1,-2,-3]})

print(sess.run([W, b]))


[array([-0.9999969], dtype=float32), array([ 0.99999082], dtype=float32)]


## The Complete Program

In [19]:
import numpy as np
import tensorflow as tf

# Model parameters
W = tf.Variable([.3], tf.float32)
b = tf.Variable([-.3], tf.float32)

# Model input and output
x = tf.placeholder(tf.float32)
linear_model = W * x + b
y = tf.placeholder(tf.float32)

# loss
loss = tf.reduce_sum(tf.square(linear_model - y)) # sum of the squares

# optimizer
optimizer = tf.train.GradientDescentOptimizer(0.01)
train = optimizer.minimize(loss)

# training data
x_train = [1,2,3,4]
y_train = [0,-1,-2,-3]

# training loop
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init) # reset values to wrong

for i in range(1000):
  sess.run(train, {x:x_train, y:y_train})

# evaluate training accuracy
curr_W, curr_b, curr_loss  = sess.run([W, b, loss], {x:x_train, y:y_train})
print("W: %s b: %s loss: %s"%(curr_W, curr_b, curr_loss))


W: [-0.9999969] b: [ 0.99999082] loss: 5.69997e-11


## tf.contrib.learn

tf.contrib.learn is a high-level TensorFlow library that simplifies the mechanics of machine learning, including the following:

    running training loops
    running evaluation loops
    managing data sets
    managing feeding

tf.contrib.learn defines many common models. 

In [22]:
import tensorflow as tf
import numpy as np

# This gets rid of a lot of WARNINGs which are rather trivial
tf.logging.set_verbosity(tf.logging.ERROR)

# Declare list of features. We only have one real-valued feature. There are many
# other types of columns that are more complicated and useful.
features = [tf.contrib.layers.real_valued_column("x", dimension=1)]

# An estimator is the front end to invoke training (fitting) and evaluation
# (inference). There are many predefined types like linear regression,
# logistic regression, linear classification, logistic classification, and
# many neural network classifiers and regressors. The following code
# provides an estimator that does linear regression.
estimator = tf.contrib.learn.LinearRegressor(feature_columns=features)

# TensorFlow provides many helper methods to read and set up data sets.
# Here we use `numpy_input_fn`. We have to tell the function how many batches
# of data (num_epochs) we want and how big each batch should be.
x = np.array([1., 2., 3., 4.])
y = np.array([0., -1., -2., -3.])
input_fn = tf.contrib.learn.io.numpy_input_fn({"x":x}, y, batch_size=4, num_epochs=1000)

# We can invoke 1000 training steps by invoking the `fit` method and passing the
# training data set.
estimator.fit(input_fn=input_fn, steps=1000)

# Here we evaluate how well our model did. In a real example, we would want
# to use a separate validation and testing data set to avoid overfitting.
print(estimator.evaluate(input_fn=input_fn))


{'loss': 3.5343056e-08, 'global_step': 1000}


## Custom Model

Suppose we wanted to create a custom model that is not built into TensorFlow. We can still retain the high level abstraction of data set, feeding, training, etc. of tf.contrib.learn. 

To define a custom model that works with tf.contrib.learn, we need to use tf.contrib.learn.Estimator. tf.contrib.learn.LinearRegressor is actually a sub-class of tf.contrib.learn.Estimator. Instead of sub-classing Estimator, we simply provide Estimator a function model_fn that tells tf.contrib.learn how it can evaluate predictions, training steps, and loss.

In [ ]:
import numpy as np
import tensorflow as tf

# Declare list of features, we only have one real-valued feature
def model(features, labels, mode):
  
  # Build a linear model and predict values
  W = tf.get_variable("W", [1], dtype=tf.float64)
  b = tf.get_variable("b", [1], dtype=tf.float64)
  y = W*features['x'] + b

  # Loss sub-graph
  loss = tf.reduce_sum(tf.square(y - labels))

  # Training sub-graph
  global_step = tf.train.get_global_step()
  optimizer = tf.train.GradientDescentOptimizer(0.01)
  train = tf.group(optimizer.minimize(loss),
                   tf.assign_add(global_step, 1))

  # ModelFnOps connects subgraphs we built to the
  # appropriate functionality.
  return tf.contrib.learn.ModelFnOps(
      mode=mode, predictions=y,
      loss=loss,
      train_op=train)

estimator = tf.contrib.learn.Estimator(model_fn=model)

# define our data set
x = np.array([1., 2., 3., 4.])
y = np.array([0., -1., -2., -3.])
input_fn = tf.contrib.learn.io.numpy_input_fn({"x": x}, y, 4, num_epochs=1000)

# train
estimator.fit(input_fn=input_fn, steps=1000)

# evaluate our model
print(estimator.evaluate(input_fn=input_fn, steps=10))
